In [ ]:
import pandas as pd
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

train = pd.read_csv('/content/drive/MyDrive/dataset/train.csv')

def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x


def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'isnt':'is not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium'

                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text
    
train["comment_text"] = train["comment_text"].progress_apply(lambda x: clean_text(x))
train["comment_text"] = train["comment_text"].progress_apply(lambda x: clean_numbers(x))
train["comment_text"] = train["comment_text"].progress_apply(lambda x: replace_typical_misspell(x))
train['comment_text'] = train['comment_text'].progress_apply(lambda x: clean_contractions(x, contraction_mapping))

<ipython-input-1-9bc4155f1eb4>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


  0%|          | 0/1804874 [00:00<?, ?it/s]

  0%|          | 0/1804874 [00:00<?, ?it/s]

  0%|          | 0/1804874 [00:00<?, ?it/s]

  0%|          | 0/1804874 [00:00<?, ?it/s]

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

train['comment_text'] = train['comment_text'].astype(str)

# List all identities
identity_columns = ['male', 'female', 'homosexual_gay_or_lesbian','transgender','heterosexual','bisexual','christian', 'jewish','muslim', 'black', 'white', 'psychiatric_or_mental_illness','hindu','buddhist','atheist','other_religion','asian','latino','other_race_or_ethnicity','intellectual_or_learning_disability']
TOXICITY_COLUMN = 'target' 

# Converting taget and identity columns to booleans
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, 1, 0)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

train_embd = convert_dataframe_to_bool(train)

SUBGROUP_AUC ='subgroup_auc'
BPSN_AUC ='bpsn_auc'
BNSP_AUC ='bnsp_auc'
def compute_bias_metrics_for_model(dataset,subgroups,model,label_col,include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model to get subgroup_auc ,bspn_auc & bpsn_auc"""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]==1])# so it picks the subgroup which are having toxic value(True)  eg subgroup='male' so we pick 'male'==True
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)
    
def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]==1] # that helps in judgeing the impact of particular positive identity column in given comment
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])
    
def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan
 
 
def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]==1) & ~(df[label]==1)]
    non_subgroup_positive_examples = df[~(df[subgroup]==1) & (df[label]==1)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[model_name])
   

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]==1) & (df[label]==1)]
    non_subgroup_negative_examples = df[~(df[subgroup]==1) & ~(df[label]==1)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[model_name])
    
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)

In [ ]:
y = train_embd['target'].values
X = train_embd.drop('target',axis=1)
X_train, X_temp, y_train, y_temp = train_test_split(X,y,test_size=0.20,random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp,y_temp,test_size=0.50,random_state=42)
y_train = tf.keras.utils.to_categorical(y_train,num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test,num_classes=2)
y_val = tf.keras.utils.to_categorical(y_val,num_classes=2)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 182 kB 64.6 MB/s 
     |████████████████████████████████| 7.6 MB 52.8 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import transformers as tfm
from transformers import BertTokenizerFast


model_class, tokenizer_class, pretrained_weights = (tfm.DistilBertModel,tfm.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
from keras.utils import pad_sequences
#from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 150

tr_sentences = X_train['comment_text'].values
train_input_ids = []
for sent in tr_sentences:
    encoded_sent = tokenizer.encode(sent,add_special_tokens = True)
    train_input_ids.append(encoded_sent)

train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
                          
te_sentences = X_test['comment_text'].values
test_input_ids = []
for sent1 in te_sentences:
    encoded_sent1 = tokenizer.encode(sent1,add_special_tokens = True)
    test_input_ids.append(encoded_sent1)


test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
                          
va_sentences = X_val['comment_text'].values
val_input_ids = []
for sent2 in va_sentences:
    encoded_sent2 = tokenizer.encode(sent2,add_special_tokens = True)
    val_input_ids.append(encoded_sent2)

val_input_ids = pad_sequences(val_input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

In [ ]:
train_attention_masks = []
for sent in train_input_ids:
    tr_att_mask = [int(token_id1 > 0) for token_id1 in sent]
    train_attention_masks.append(tr_att_mask)
    
test_attention_masks = []
for sent1 in test_input_ids:
    te_att_mask = [int(token_id2 > 0) for token_id2 in sent1]
    test_attention_masks.append(te_att_mask)
    
val_attention_masks = []
for sent2 in val_input_ids:
    val_att_mask = [int(token_id3 > 0) for token_id3 in sent2]
    val_attention_masks.append(val_att_mask)

In [ ]:
# model_construction

from transformers import  DistilBertConfig,TFDistilBertModel
import tensorflow as tf
from tensorflow import keras

distil_bert = 'distilbert-base-uncased'
config = DistilBertConfig(dropout=0.0, attention_dropout=0.0)
config.output_hidden_states = False
transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)

input_ids_in = tf.keras.layers.Input(shape=(150,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(150,), name='masked_token', dtype='int32') 
embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
layer = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNGRU(150,return_sequences=True))(embedding_layer)
layer = tf.keras.layers.Dropout(0.5)(layer)
layer = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(70))(layer)
layer = tf.keras.layers.Dense(32)(layer)
layer = tf.keras.layers.Activation('sigmoid')(layer)
layer = tf.keras.layers.Dense(2,name='out_layer')(layer)
layer = tf.keras.layers.Activation('sigmoid')(layer)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = layer)

for layer in model.layers[:3]:
    layer.trainable = False

In [ ]:
# # model_construction

# from transformers import  DistilBertConfig,TFDistilBertModel
# import tensorflow as tf
# from tensorflow import keras

# distil_bert = 'distilbert-base-uncased'
# config = DistilBertConfig(dropout=0.0, attention_dropout=0.0)
# config.output_hidden_states = False
# transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)

# input_ids_in = tf.compat.v1.keras.layers.Input(shape=(150,), name='input_token', dtype='int32')
# input_masks_in = tf.compat.v1.keras.layers.Input(shape=(150,), name='masked_token', dtype='int32') 
# embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
# layer = tf.compat.v1.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNGRU(150,return_sequences=True))(embedding_layer)
# layer = tf.compat.v1.keras.layers.Dropout(0.5)(layer)
# layer = tf.compat.v1.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(70))(layer)
# layer = tf.compat.v1.keras.layers.Dense(32)(layer)
# layer = tf.compat.v1.keras.layers.Activation('sigmoid')(layer)
# layer = tf.compat.v1.keras.layers.Dense(2,name='out_layer')(layer)
# layer = tf.compat.v1.keras.layers.Activation('sigmoid')(layer)
# model = tf.compat.v1.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = layer)

# for layer in model.layers[:3]:
#     layer.trainable = False

In [ ]:
train_attention_masks = np.array(train_attention_masks)
val_attention_masks = np.array(val_attention_masks)
print(type(val_attention_masks)) 

In [ ]:
#change batch_size=512 to 100 with epochs 2 to 1, crash issues will be solved
def scheduler(epoch, lr):
    lr = lr*(0.68**epoch)
    return lr
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
 
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4, min_delta=0.02, restore_best_weights=True),
    tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=2e-5, patience=2, verbose=0, mode='auto', min_delta=0.001, cooldown=0, min_lr=2e-5)
]

hist = model.fit(x=[train_input_ids,train_attention_masks],y=y_train, epochs=1,batch_size=100,validation_data=([val_input_ids,val_attention_masks],y_val),callbacks=callbacks)


In [ ]:
test_attention_masks = np.array(test_attention_masks)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn import metrics

predict_val = model.predict([test_input_ids,test_attention_masks])


In [ ]:
identity_columns = ['male', 'female', 'homosexual_gay_or_lesbian','transgender','heterosexual','bisexual','christian', 'jewish','muslim', 'black', 'white', 'psychiatric_or_mental_illness','hindu','buddhist','atheist','other_religion','asian','latino','other_race_or_ethnicity']

TOXICITY_COLUMN = 'target'
X_test['target']= y_test[:,1]
MODEL_NAME = 'bert_lstm'
X_test[MODEL_NAME] = list(predict_val[:,1]) # so we check for toxic comment
bias_metrics_df = compute_bias_metrics_for_model(X_test, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
bias_metrics_df

In [ ]:
import numpy as np
bert_lstm_model=get_final_metric(bias_metrics_df, calculate_overall_auc(X_test,MODEL_NAME))
bert_lstm_model